In [36]:
!pip install --upgrade dash
!pip install dash-bootstrap-components
!pip install rasterio fiona shapely pyproj
!pip install -U scikit-fuzzy
!pip install ipympl
!pip install pycrs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.9/202.9 kB 3.8 MB/s eta 0:00:00


In [7]:
from dash import Dash, dash_table, html, dcc, callback, Output, Input
import plotly.express as px
import pandas as pd
import geopandas as gpd
import numpy as np

In [65]:
# Import packages
from dash import Dash, html, dash_table, dcc, callback, Output, Input
import pandas as pd
import plotly.express as px
import dash_bootstrap_components as dbc

# Load data
fields2024 = gpd.read_file('/content/fields2024.shp')
meanSAR = gpd.read_file('/content/VH_mean2024.csv')
stdSAR = gpd.read_file('/content/VH_stdDev2024.csv')

melted_df = meanSAR.drop(columns = ['field_1']).melt(id_vars=['Name', 'Status', 'geometry'], var_name='Date', value_name='VH_mean')

melted_df['Date'] = pd.to_datetime(melted_df['Date'])
melted_df['VH_mean'] = pd.to_numeric(melted_df['VH_mean'], errors='coerce')
# Get the bounds of the GeoDataFrame
bounds = fields2024.total_bounds

# Calculate the center coordinates
center_lat = (bounds[1] + bounds[3]) / 2
center_lon = (bounds[0] + bounds[2]) / 2

# Initialize the app - incorporate a Dash Bootstrap theme
external_stylesheets = [dbc.themes.CERULEAN]
app = Dash(__name__, external_stylesheets=external_stylesheets)

# App layout
app.layout = dbc.Container([
    dbc.Row([
        html.Div('WxC Subak Monitoring', className="text-primary text-center fs-3")
    ]),

    dbc.Row([
        dbc.RadioItems(options=[{"label": x, "value": x} for x in ['WINs', 'Size', 'Productivity']],
                       value='WINs',
                       inline=True,
                       id='radio-buttons-final')
    ]),

    dbc.Row([
        dbc.Col([
            dash_table.DataTable(data=fields2024.drop(columns = ['geometry', 'Descriptio']).to_dict('records'), page_size=10)
        ], width=6),

        dbc.Col([
            dcc.Graph(id="graph")
        ], width=6),
    ]),
    dbc.Row([
        dcc.Graph(id="time-series")

    ]),


], fluid=True)

# Add controls to build the interaction
@callback(
    Output(component_id='graph', component_property='figure'),
    Input(component_id='radio-buttons-final', component_property='value')
)

def update_graph(field):
    fig = px.choropleth_mapbox(
        fields2024,
        geojson=fields2024.geometry,
        locations=fields2024.index,  # Or use a column containing location names
        color="Status",  # Replace with the column you want to visualize
        color_continuous_scale="Viridis",  # Choose a color scale
        mapbox_style="carto-positron",  # Choose a mapbox style
        center={"lat": center_lat, "lon": center_lon},  # Adjust center coordinates
        zoom=15,  # Adjust zoom level
    )
    fig.update_layout(
        margin={"r":0,"t":0,"l":0,"b":0})

    return fig

@callback(
    Output(component_id='time-series', component_property='figure'),
    Input(component_id='radio-buttons-final', component_property='value')
)
def update_timeSeries(field):
    fig = px.line(melted_df[melted_df['Name']=='Palin'], x='Date', y='VH_mean', title='Mean SAR Time Series')
    fig.show()

    return fig

# Run the app
if __name__ == '__main__':
    app.run(debug=True)

<IPython.core.display.Javascript object>